In [5]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from statistics import mean
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import itertools
import joblib
from matplotlib import pyplot as plot
import os
import gc
from load_dataset import get_dataloader


In [2]:
class ConvNet(nn.Module):
    def __init__(self, in_channels, max_seq_length, input_dim):
        super(ConvNet, self).__init__()
        
        self.num_conv_layers = len(in_channels)
        self.max_seq_length = max_seq_length
        unique_aa = 21
        # Convolutional layers
        self.layers = nn.ModuleList()
        in_channels = [1, *in_channels]  # Input channel for the first layer
        self.conv_out_height, self.conv_out_width = input_dim

        for i in range(self.num_conv_layers):
            self.layers.append(nn.Conv2d(in_channels[i], in_channels[i+1], kernel_size=2, stride=1, padding=1))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            self.conv_out_height = self.compute_dim(self.conv_out_height, 2, 1, 1,2, 2)
            self.conv_out_width = self.compute_dim(self.conv_out_width, 2, 1, 1,2, 2)
        self.conv_net = nn.Sequential(*self.layers)

        # Fully connected layer
        self.fc = nn.Linear(int(in_channels[-1] * self.conv_out_height * self.conv_out_width), max_seq_length* unique_aa)
    def forward(self, x):
        # Reshape the input to match the expected input shape (batch_size, channels, height, width)
        x = x.unsqueeze(1)  # Add a channel dimension (batch_size, 1, height, width)
        
        # Convolutional layers
        x = self.conv_net(x)
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        
        # Fully connected layer
        x = self.fc(x)
        
        return x
    def compute_dim(self, dim, conv_k, conv_s, conv_p, pool_k, pool_s):
        dim = np.floor(((dim - conv_k + 2 *conv_p)/conv_s)+1)
        return np.floor(((dim - pool_k)/pool_s)+1)


In [3]:
gpu = torch.device('cpu') 
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu

device(type='cuda', index=0)

In [ ]:
train_loader, test_loader = get_dataloader('Prots_embeddings_mat.joblib', 128)

In [4]:
def train(conv_net, train_loader, test_loader, optimizer:optim, epochs:int = 10, batch_size:int = 64, loss_fn =nn.MSELoss()):
    train_losses = []
    test_losses = []

    for i in range(epochs):
        print('epoch {}'.format(i))
        epoch_loss = []
        test_loss = []

        # Training
        for _, (batch,labels) in tqdm(enumerate(train_loader), total=np.ceil(len(train_loader)/batch_size)):
            conv_net.zero_grad()
            # print(type(batch), batch.shape)
            batch = batch.to(gpu)
            labels = labels.view(len(batch), -1).to(gpu)
            
            output = conv_net(batch)
            loss_train = loss_fn(output, labels)
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
            epoch_loss.append(loss_train.item())

        # Test Set
        for _, (batch,labels) in enumerate(test_loader):
            conv_net.zero_grad()
            batch = batch.to(gpu)
            labels = labels.view(len(batch), -1).to(gpu)

            output = conv_net(batch)
            loss_test = loss_fn(output, labels)
            test_loss.append(loss_test.item())

        train_losses.append(np.average(epoch_loss))
        test_losses.append(np.average(test_loss))
    return train_losses, test_losses
    


In [11]:
data = torch.randn(4,300,1024)

In [18]:
net = ''
del net 
gc.collect()

16

In [ ]:
conv_layers = [4,8]  # Define the number of convolutional layers
net = ConvNet(conv_layers, 1965, data.shape[1:])

In [ ]:
optimizer = optim.Adagrad(net.parameters(), lr=0.001)

In [ ]:
results = train(net, train_loader, test_loader, optimizer, epochs=100)